In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import math
import sys
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
import config_origins as config

# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_dataoutput = config.path_dataoutput
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_HAID_date = config.dico_HAID_date
path_extracted_HA = config.path_extracted_HA

change the configuration file if not done yet!


# Download variables

In [7]:
li_df = []
for path_ in glob.glob(os.path.join(path_dataoutput, 'HA', 'HA*', 'HA*.csv')):
    HAID = path_.split('\\')[-1].split('.')[0]
    print('---------------------', HAID)
    df_ = pd.read_csv(path_, sep=';',encoding='latin1') 
    df_['HAID'] = HAID
    df_['date'] = dico_HAID_date[HAID]
    df_ = df_.rename(columns={'Pen ':'Pen'})
    df_ = df_[~df_['FocalLegringName'].isnull()]
    print(df_.shape)
    #print(df_.columns)
    #display(df_.head(2))
    li_df.append(df_)
df_HA = pd.concat(li_df)
#small typo corrections
df_HA['BirdType'] = df_HA['BirdType'].fillna('normal')
df_HA['BirdType'] = df_HA['BirdType'].map(lambda x: x.strip(' '))
df_HA = df_HA.rename(columns={'Pen':'PenID'})
df_HA['PenID'] = df_HA['PenID'].map(lambda x: 'pen'+str(int(x)))
print(df_HA.columns)
print(df_HA.shape)
df_HA.head(3)

--------------------- HA1
(257, 20)
--------------------- HA2
(256, 30)
--------------------- HA3


C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,3,6,15,16,17,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(260, 30)
--------------------- HA4


C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,3,6,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(273, 30)
Index(['Person_toe', 'Time', 'PenID', 'FocalLegringName', 'BirdType',
       'CombPecks', 'Toes', 'RFPododermatitis', 'RFBumblefoot', 'RFinjuries',
       'LFPododermatitis', 'LFBumblefoot', 'LFinjuries', 'Reliability_MC',
       'PassageID_toe', 'Diarrhea', 'Commentaires', 'SomeHenID', 'HAID',
       'date', 'ï»¿Person_toe', 'Claw', 'Person_feather', 'Reliability_SS',
       'PassageID_feather', 'NeckFeather', 'BreastFeather', 'CloacaFeather',
       'BackFeather', 'WingFeather', 'TailFeather', 'ï»¿Time'],
      dtype='object')
(1046, 32)


,Person_toe,Time,PenID,FocalLegringName,BirdType,CombPecks,Toes,RFPododermatitis,RFBumblefoot,RFinjuries,...,Person_feather,Reliability_SS,PassageID_feather,NeckFeather,BreastFeather,CloacaFeather,BackFeather,WingFeather,TailFeather,ï»¿Time
0,M,16:15,pen3,1green,normal,0.0,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M,13:59,pen5,10green,epi,0.0,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M,16:14,pen3,11grey,normal,88.0,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Processing

In [8]:
#add variables
#overall pododeratitis: should be worst over time, so only HA4
df_HA['Injuries'] = df_HA.apply(lambda x: x['LFinjuries']+x['RFinjuries'], axis=1)
df_HA['Pododermatitis'] = df_HA.apply(lambda x: x['RFPododermatitis']+x['LFPododermatitis'], axis=1)
df_HA['Bumblefoot'] = df_HA.apply(lambda x: x['RFBumblefoot']+x['LFBumblefoot'], axis=1)
df_HA['Toes'] = df_HA['Toes'].fillna('')
df_HA['Toes_inflammed'] = df_HA['Toes'].map(lambda x: 'inflammed' in x.lower())
#df_HA['BigInjuriesOrInflammed'] = df_HA.apply(lambda x: 1 if ((x['LFinjuries']>=100) | (x['RFinjuries']>=100) |\
#                                                              (x['Toes_inflammed']==1)) else 0, axis=1)
df_HA['Diarrhea'] = df_HA['Diarrhea'].replace('N/A',np.nan).replace('N/A ',np.nan)
df_HA['Diarrhea'] = df_HA['Diarrhea'].astype(float)
df_HA['Diarrhea'] = df_HA['Diarrhea'].map(lambda x: 1 if x==2 else x)
#overall feather as in general have the same tendency
df_HA['Feathers'] = df_HA.apply(lambda x: np.mean([x['BreastFeather'],x['CloacaFeather'],
                                                   x['WingFeather'],x['TailFeather']]), axis=1)
#backfeather & NeckFeather removed as not reliable at all
df_HA.head(3)

,Person_toe,Time,PenID,FocalLegringName,BirdType,CombPecks,Toes,RFPododermatitis,RFBumblefoot,RFinjuries,...,CloacaFeather,BackFeather,WingFeather,TailFeather,ï»¿Time,Injuries,Pododermatitis,Bumblefoot,Toes_inflammed,Feathers
0,M,16:15,pen3,1green,normal,0.0,,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,False,NaN
1,M,13:59,pen5,10green,epi,0.0,,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,False,NaN
2,M,16:14,pen3,11grey,normal,88.0,,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,False,NaN


display(df_HA['BigInjuriesOrInflammed'].value_counts())
display(df_HA['Toes'].value_counts())
display(df_HA['Claw'].value_counts())

In [9]:
#initialize set of variables
#keep LeftFoot and RightFoot separate due to slightly different conditions that might influence the realiability (e.g. 2 
#legrings on one side, e.g. how the students hold the birds)
li_cont_T = ['CombPecks', 'RFPododermatitis', 'RFinjuries', 'LFPododermatitis', 'LFinjuries','Injuries','Pododermatitis']
li_cat_T = ['Diarrhea','RFBumblefoot', 'LFBumblefoot', 'Bumblefoot']
li_cont_F = ['Feathers','NeckFeather', 'BreastFeather', 'CloacaFeather', 'BackFeather', 'WingFeather','TailFeather']

# Toes

In [10]:
df_HA_T = df_HA[(df_HA['Reliability_MC']==1)&(~df_HA['Person_toe'].isnull())&(~df_HA['PassageID_toe'].isnull())].copy()
print(df_HA_T.shape)
df_HA_T.head(3)

(344, 37)


,Person_toe,Time,PenID,FocalLegringName,BirdType,CombPecks,Toes,RFPododermatitis,RFBumblefoot,RFinjuries,...,CloacaFeather,BackFeather,WingFeather,TailFeather,ï»¿Time,Injuries,Pododermatitis,Bumblefoot,Toes_inflammed,Feathers
14,C,10:07,pen4,17orange,normal,75.0,,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,False,NaN
15,C,10:45,pen4,17orange,normal,100.0,,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,False,NaN
16,M,10:04,pen4,17orange,normal,25.0,,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,False,NaN


In [11]:
########## INTER: mark vs camille
#verify each value of "ID4Rea" have one observation for the two raters of the feathers and the two raters for the toes
#unique ID for each observations based on their unique set of info
df_HA_T_inter = df_HA_T.copy()
df_HA_T_inter['ID4Rea'] = df_HA_T_inter.apply(lambda x: (x['PenID'].strip(' ')+'_'+x['HAID'].strip(' ')+'_'+\
                                             x['FocalLegringName'].strip(' ')+'_'+str(int(x['PassageID_toe']))), axis=1)
df_HA_T_inter = df_HA_T_inter[['Person_toe','ID4Rea']+li_cont_T+li_cat_T]
#correct persons name
df_HA_T_inter['Person_toe'] = df_HA_T_inter['Person_toe'].map(lambda x: x[0])
display(df_HA_T_inter['Person_toe'].value_counts())

################# save for CCC in R
li_df = []
for v in li_cont_T+li_cat_T:
    df_ = df_HA_T_inter.pivot(index='ID4Rea', columns='Person_toe', values=v).reset_index()
    df_['HAvariable'] = v
    li_df.append(df_)
df_HA_T_inter_pivot = pd.concat(li_df)
print(df_HA_T_inter_pivot.shape)
#save for CCC in R
df_HA_T_inter_pivot.to_csv(os.path.join(path_extracted_HA ,'df_HA_T_inter_pivot.csv'), index=False, sep=';')
display(df_HA_T_inter_pivot.head(3))

#identify potential issues
#df_ = df_HA_T_inter.groupby(['ID4Rea'])['Person_toe'].agg(lambda x: '_'.join(sorted(list(x)))).reset_index()
#df_[df_['Person_toe']!='C_M']

C    172
M    172
Name: Person_toe, dtype: int64

(1892, 4)


Person_toe,ID4Rea,C,M,HAvariable
0,pen10_HA3_18orange_1,25.0,10.0,CombPecks
1,pen10_HA3_18orange_2,25.0,0.0,CombPecks
2,pen10_HA3_62orange_1,0.0,0.0,CombPecks


In [12]:
########## INTRA: mark vs mark AND camille vs camille
#verify each value of "ID4Rea" have one observation for the two raters of the feathers and the two raters for the toes
#unique ID for each observations based on their unique set of info
df_HA_T_intra = df_HA_T.copy()
df_HA_T_intra['ID4Rea'] = df_HA_T_intra.apply(lambda x: (x['PenID'].strip(' ')+'_'+x['HAID'].strip(' ')+'_'+\
                                             x['FocalLegringName'].strip(' ')), axis=1)
df_HA_T_intra = df_HA_T_intra[['Person_toe','ID4Rea', 'PassageID_toe']+li_cont_T+li_cat_T]
#correct persons name
df_HA_T_intra['Person_toe'] = df_HA_T_intra['Person_toe'].map(lambda x: x[0])
display(df_HA_T_intra['Person_toe'].value_counts())
#################3 save for CCC in R
li_df = []
for v in li_cont_T+li_cat_T:
    df_ = df_HA_T_intra.pivot(index=['ID4Rea','Person_toe'], columns='PassageID_toe', values=v).reset_index()
    df_['HAvariable'] = v
    li_df.append(df_)
df_HA_T_intra_pivot = pd.concat(li_df)
print(df_HA_T_intra_pivot.shape)
#save for CCC in R
df_HA_T_intra_pivot.to_csv(os.path.join(path_extracted_HA ,'df_HA_T_intra_pivot.csv'), index=False, sep=';')
display(df_HA_T_intra_pivot.head(3))

#identify potential issues
#df_ = df_HA_T_intra.groupby(['ID4Rea'])['Person_toe'].agg(lambda x: '_'.join(sorted(list(x)))).reset_index()
#df_[df_['Person_toe']!='C_M']

C    172
M    172
Name: Person_toe, dtype: int64

(1892, 5)


PassageID_toe,ID4Rea,Person_toe,1.0,2.0,HAvariable
0,pen10_HA3_18orange,C,25.0,25.0,CombPecks
1,pen10_HA3_18orange,M,10.0,0.0,CombPecks
2,pen10_HA3_62orange,C,0.0,0.0,CombPecks


# Feathers

In [13]:
df_HA_F = df_HA[(df_HA['Reliability_SS']==1)&(~df_HA['Person_feather'].isnull())&(~df_HA['PassageID_feather'].isnull())].copy()
print(df_HA_F.shape)
df_HA_F.head(3)

(348, 37)


,Person_toe,Time,PenID,FocalLegringName,BirdType,CombPecks,Toes,RFPododermatitis,RFBumblefoot,RFinjuries,...,CloacaFeather,BackFeather,WingFeather,TailFeather,ï»¿Time,Injuries,Pododermatitis,Bumblefoot,Toes_inflammed,Feathers
0,NaN,08:20,pen3,1green,normal,25.0,,0.0,0.0,50.0,...,100.0,100.0,97.0,97.0,NaN,50.0,0.0,0.0,False,98.50
1,NaN,08:20,pen3,30grey,normal,25.0,,0.0,0.0,50.0,...,100.0,100.0,100.0,100.0,NaN,50.0,0.0,0.0,False,92.75
3,NaN,08:20,pen3,36green,normal,140.0,,50.0,0.0,0.0,...,100.0,100.0,66.0,100.0,NaN,0.0,100.0,0.0,False,85.00


In [14]:
########## INTER: sabine vs Satar
#verify each value of "ID4Rea" have one observation for the two raters of the feathers and the two raters for the feathers
#unique ID for each observations based on their unique set of info
df_HA_F_inter = df_HA_F.copy()
df_HA_F_inter['ID4Rea'] = df_HA_F_inter.apply(lambda x: (x['PenID'].strip(' ')+'_'+x['HAID'].strip(' ')+'_'+\
                                             x['FocalLegringName'].strip(' ')+'_'+str(int(x['PassageID_feather']))), axis=1)
df_HA_F_inter = df_HA_F_inter[['Person_feather','ID4Rea']+li_cont_F]
#correct persons name
df_HA_F_inter['Person_feather'] = df_HA_F_inter['Person_feather'].map(lambda x: x.lower())
display(df_HA_F_inter['Person_feather'].value_counts())
#identify potential issues
#df_ = df_HA_F_inter.groupby(['ID4Rea'])['Person_feather'].agg(lambda x: '_'.join(sorted(list(x)))).reset_index()
#df_[df_['Person_feather']!='sabine_satar']
################# save for CCC in R
li_df = []
for v in li_cont_F:
    df_ = df_HA_F_inter.pivot(index='ID4Rea', columns='Person_feather', values=v).reset_index()
    df_['HAvariable'] = v
    li_df.append(df_)
df_HA_F_inter_pivot = pd.concat(li_df)
print(df_HA_F_inter_pivot.shape)
#save for CCC in R
df_HA_F_inter_pivot.to_csv(os.path.join(path_extracted_HA ,'df_HA_F_inter_pivot.csv'), index=False, sep=';')
display(df_HA_F_inter_pivot.head(3))

sabine    174
satar     174
Name: Person_feather, dtype: int64

(1218, 4)


Person_feather,ID4Rea,sabine,satar,HAvariable
0,pen10_HA2_18orange_1,100.0,99.5,Feathers
1,pen10_HA2_18orange_2,100.0,100.0,Feathers
2,pen10_HA2_39black_1,93.0,100.0,Feathers


In [15]:
########## INTRA: sabine vs Sabine / Satar vs Satar
#verify each value of "ID4Rea" have one observation for the two raters of the feathers and the two raters for the feathers
#unique ID for each observations based on their unique set of info
df_HA_F_intra = df_HA_F.copy()
df_HA_F_intra['ID4Rea'] = df_HA_F_intra.apply(lambda x: (x['PenID'].strip(' ')+'_'+x['HAID'].strip(' ')+'_'+\
                                             x['FocalLegringName'].strip(' ')), axis=1)
df_HA_F_intra = df_HA_F_intra[['Person_feather','ID4Rea','PassageID_feather']+li_cont_F]
#correct persons name
df_HA_F_intra['Person_feather'] = df_HA_F_intra['Person_feather'].map(lambda x: x.lower())
display(df_HA_F_intra['Person_feather'].value_counts())
################### save for CCC in R
li_df = []
for v in li_cont_F:    
    df_ = df_HA_F_intra.pivot(index=['ID4Rea','Person_feather'], columns='PassageID_feather', values=v).reset_index()
    df_['HAvariable'] = v
    li_df.append(df_)
df_HA_F_intra_pivot = pd.concat(li_df)
print(df_HA_F_intra_pivot.shape)
#save for CCC in R
df_HA_F_intra_pivot.to_csv(os.path.join(path_extracted_HA ,'df_HA_F_intra_pivot.csv'), index=False, sep=';')
display(df_HA_F_intra_pivot.head(3))

sabine    174
satar     174
Name: Person_feather, dtype: int64

(1218, 5)


PassageID_feather,ID4Rea,Person_feather,1.0,2.0,HAvariable
0,pen10_HA2_18orange,sabine,100.0,100.00,Feathers
1,pen10_HA2_18orange,satar,99.5,100.00,Feathers
2,pen10_HA2_39black,sabine,93.0,97.25,Feathers
